### Combining Datasets: Concat and Append
* Concatenation function: pd.concat
    * Concatenate pandas objects along a particular axis with optional set logic
along the other axes
    * pd.concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False, keys=None, levels=None, names=None, verify_integrity=False, copy=True)
        * objs : a sequence or mapping of Series, DataFrame, or Panel objects
            *If a dict is passed, the sorted keys will be used as the `keys` argument, unless it is passed, in which case the values will be selected (see below). Any None objects will be dropped silently unless they are all None in which case a ValueError will be raised
        * axis : {0/'index', 1/'columns'}, default 0
            * The axis to concatenate along
        * join : {'inner', 'outer'}, default 'outer'
            * How to handle indexes on other axis(es)
        * join_axes : list of Index objects
            * Specific indexes to use for the other n - 1 axes instead of performing inner/outer set logic
        * ignore_index : boolean, default False
            * If True, do not use the index values along the concatenation axis. 
            * The resulting axis will be labeled 0, ..., n - 1. This is useful if you are concatenating objects where the concatenation axis does not have meaningful indexing information. Note the index values on the other axes are still respected in the join.
        * keys : sequence, default None
            * If multiple levels passed, should contain tuples. Construct hierarchical index using the passed keys as the outermost level
        * levels : list of sequences, default None
            * Specific levels (unique values) to use for constructing a MultiIndex. Otherwise they will be inferred from the keys
        * names : list, default None
            * Names for the levels in the resulting hierarchical index
        * verify_integrity : boolean, default False
            * Check whether the new concatenated axis contains duplicates. This can be very expensive relative to the actual data concatenation
        * copy : boolean, default True
            * If False, do not copy data unnecessarily

In [1]:
import pandas as pd
import numpy as np

In [ ]:
def make_df(cols, ind):
    """Quickly make a DataFrame"""
    data = {c: [str(c) + str(i) for i in ind]
           for c in cols}
    return pd.DataFrame(data, ind)

In [ ]:
# example of DataFrame
make_df('ABC', range(3))

### Recall: Concatenation of NumPy Arrays

In [ ]:
# combine the contents of two or more arrays into a single array:
x = [1, 2, 3]
y = [4, 5, 6]
z = [7, 8, 9]
np.concatenate([x, y, z])

In [ ]:
# it takes an axis keyword 
# that allows you to specify the axis 
# along which the result will be concatenated:
x = [[1, 2],
     [3, 4]]
np.concatenate([x, x], axis=1)

### Simple Concetenation with pd.concat
* **pd.concat** (objs, axis=0, join='outer', join_axes=None, ignore_index=False,
keys=None, levels=None, names=None, verify_integrity=False,
copy=True)

In [ ]:
# simple concatenations of arrays
ser1 = pd.Series(['A', 'B', 'C'], index=[1, 2, 3])
ser2 = pd.Series(['D', 'E', 'F'], index=[4, 5, 6])
pd.concat([ser1, ser2])

In [ ]:
# concatenate higher-dimensional objects, such as DataFrames
df1 = make_df('AB', [1, 2])
df2 = make_df('AB', [3, 4])
print(df1); print(df2); print(pd.concat([df1, df2]))

In [ ]:
df3 = make_df('AB', [0, 1])
df4 = make_df('CD', [0, 1])
print(df3); print(df4); print(pd.concat([df3, df4], axis = 'columns'))

#### Duplicate indices
* Padans concetenation *preserves indices*, even if the result will have duplicae indices.

In [ ]:
x = make_df('AB', [0, 1])
y = make_df('AB', [2, 3])
y.index = x.index # make duplicate indices!
print(x); print(y);print(pd.concat([x, y]))

In [ ]:
print(pd.concat([x, y], ignore_index=True))

#### Catching the repeats as an error
* If you’d like to simply verify that the indices in the result of pd.concat() do not overlap, you can specify the verify_integrity flag.
* With this set to True, the concatenation will raise an exception if there are duplicate indices.

In [ ]:
try:
    pd.concat([x, y], verify_integrity=True)
except ValueError as e:
    print("ValueError", e)

#### Ignoring the index
* ignore_index : boolean, default False
    * If True, do not use the index values along the concatenation axis.
    * The resulting axis will be labeled 0, ..., n - 1. 
    * This is useful if you are concatenating objects where the concatenation axis does not have meaningful indexing information. 
    * Note the index values on the other axes are still respected in the join.

In [ ]:
# Ignoring the indexing
print(pd.concat([x, y], ignore_index=True))

#### Adding MultiIndex keys
* keys : sequence, default None
    * If multiple levels passed, should contain tuples. 
    * Construct hierarchical index using the passed keys as the outermost level

In [ ]:
print(x); print(y);print(pd.concat([x, y], keys = ['x', 'y']))

#### Concatenation with joins
* join : {'inner', 'outer'}, default 'outer'
    * How to handle indexes on other axis(es)
* join_axes : list of Index objects
    * Specific indexes to use for the other n - 1 axes instead of performing inner/outer set logic

In [ ]:
df5 = make_df('ABC', [1, 2])
df6 = make_df('BCD', [3, 4])
print(df5); print(df6);print(pd.concat([df5, df6]))

In [ ]:
print(pd.concat([df5, df6], join = 'inner'))

In [ ]:
# specify that the returned columns should be the same 
# as those of the first input
print(pd.concat([df5, df6], join_axes = [df5.columns]))


#### The append() method
* df5.append(other, ignore_index=False, verify_integrity=False)
    * Unlike the *append()* and *extend()* methods of Python lists, the *append()* method in Pandas does not modify the original - instead, it creates a new object with the combined data.
    * It also is not a very efficient method, because it involves creation of a new index and data buffer.
    * better to use concat() function

In [ ]:
# less efficient
df = pd.DataFrame(columns=['A'])
for i in range(5):
    df = df.append({'A': i}, ignore_index=True)
df

In [ ]:
# more efficient
pd.concat([pd.DataFrame([i], columns=['A']) for i in range(5)],
          ignore_index=True)

### Combining Datasets: Merge and Join
* pd.merge
    * Merge DataFrame objects by performing a database-style join operation by columns or indexes
    * If joining columns on columns, the DataFrame indexes *will be ignored*. 
    * Otherwise if joining indexes on indexes or indexes on a column or columns, the index will be passed on.

* <font color = red> pd.merge(left, right, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, sort=False, suffixes=('_x', '_y'), copy=True, indicator=False, validate=None) </font>

    * left: df
    * right: df
    * how: {'left', 'right', 'outer', 'inner'}, default 'inner'
        * left: use only keys from left frame, similar to a SQL left outer join;
          preserve key order
        * right: use only keys from right frame, similar to a SQL right outer join; preserve key order
        * outer: use union of keys from both frames, similar to a SQL full outer
          join; sort keys lexicographically
        * inner: use intersection of keys from both frames, similar to a SQL inner join; preserve the order of the left keys
    * on : label or list
        * Field names to join on. 
        * Must be found in both DataFrames. 
        * If on is None and not merging on indexes, then it merges on the intersection of the columns by default.
    * left_on : label or list, or array-like
        * Field names to join on in left DataFrame. 
        * Can be a vector or list of vectors of the length of the DataFrame to use a particular vector as the join key instead of columns
    * right_on : label or list, or array-like
        * Field names to join on in right DataFrame or vector/list of vectors per left_on docs  
    * left_index : boolean, default False
        * Use the index from the left DataFrame as the join key(s). 
        * If it is a MultiIndex, the number of keys in the other DataFrame (either the index or a number of columns) must match the number of levels
    * right_index : boolean, default False 
        * Use the index from the right DataFrame as the join key. 
        * Same caveats as left_index
    * sort : boolean, default False
        * Sort the join keys lexicographically in the result DataFrame. 
        * If False, the order of the join keys depends on the join type (how keyword)
    * suffixes : 2-length sequence (tuple, list, ...)
        * Suffix to apply to overlapping column names in the left and right
        side, respectively
    * copy : boolean, default True
        * If False, do not copy data unnecessarily
    * indicator : boolean or string, default False
        * If True, adds a column to output DataFrame called "_merge" with
        information on the source of each row.
        * If string, column with information on source of each row will be added to output DataFrame, and column will be named value of string.
        * Information column is Categorical-type and takes on a value of "left_only" for observations whose merge key only appears in 'left' DataFrame, "right_only" for observations whose merge key only appears in 'right' DataFrame, and "both" if the observation's merge key is found in both.

        .. versionadded:: 0.17.0

    * validate : string, default None
        * If specified, checks if merge is of specified type.

        * "one_to_one" or "1:1": check if merge keys are unique in both
          left and right datasets.
        * "one_to_many" or "1:m": check if merge keys are unique in left
          dataset.
        * "many_to_one" or "m:1": check if merge keys are unique in right
          dataset.
        * "many_to_many" or "m:m": allowed, but does not result in checks.

        .. versionadded:: 0.21.0

### Relational Algebra
### Categories of Joins
* *one-to-one*, *many-to-one*, *many-to-many* joins

#### One-to-one joins
* column-wise concatenation

In [ ]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})

In [ ]:
df3 = pd.merge(df1, df2)
df3
# The pd.merge() function recognizes that each DataFrame has an “employee” column,
# and automatically joins using this column as a key. 
# The result of the merge is a new DataFrame that combines the information 
# from the two inputs. 
# Notice that the order of entries in each column is not necessarily maintained: in this case, the order of the
# “employee” column differs between df1 and df2, 
# and the pd.merge() function correctly accounts for this. 
# Additionally, keep in mind that the merge in general 
# discards the index, except in the special case of merges by index

#### Many-to-one joins
* One of the two key columns contains duplicate entries

In [ ]:
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
'supervisor': ['Carly', 'Guido', 'Steve']})
print(df3); print(df4); print(pd.merge(df3, df4))

#### Many-to-many joins
* The key column in both the left and right array contains duplicates, then the result is a many-to-many merge

In [ ]:
df1

In [ ]:
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
'spreadsheets', 'organization']})

In [ ]:
print(df1); print(df5); print(pd.merge(df1, df5))

### Specification of the Merge Key

#### The on keyword
* on : label or list
    * Field names to join on.
    * <mark>Must be found in both DataFrames.</mark>
    * If on is None and not merging on indexes, then it merges on the intersection of the columns by default.

In [ ]:
print(df1); print(df2);print(pd.merge(df1, df2, on = 'employee'))

#### The left_on and right_on keywords
* At times you may wish to merge two datasets with different column names; 
* for example, we may have a dataset in which the employee name
* is labeled as “name” rather than “employee”. In this case,
* we can use the left_on and right_on keywords to specify the two column names
* <font color = red> drop redundant column using .drop() method. </font>

In [ ]:
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'salary': [70000, 80000, 120000, 90000]})
print(df1); print(df3);

In [ ]:
pd.merge(df1, df3, left_on = 'employee', right_on = 'name')

In [ ]:
pd.merge(df1, df3, left_on = 'employee', right_on = 'name').drop('name', axis = 1)

#### The left_index and right_index keywords
* merge on an index
* <font color = red> DataFrames implement the join() method, which performs a merge that defaults to joining on indices </font>
* <font color = red> mix indices and columns, combine **left_index with right_on** or **left_on with right_index** to get the desired behavior

In [ ]:
df1a = df1.set_index('employee')
df2a = df2.set_index('employee')

In [ ]:
df1a

In [ ]:
df2a

In [ ]:
pd.merge(df1a, df2a, left_index = True, right_index = True)

In [ ]:
# DataFrames implement the join() method, 
# which performs a merge that defaults to joining on indices
df1a.join(df2a)

In [ ]:
# mix indices and columns, combine left_index with right_on 
# or left_on with right_index to get the desired behavior
pd.merge(df1a, df3, left_index = True, right_on='name')

### Specifying Set Arithmetic for Joins
* how: {'left', 'right', 'outer', 'inner'}, default 'inner'
    * left: use only keys from left frame, similar to a SQL left outer join; preserve key order
    * right: use only keys from right frame, similar to a SQL right outer join; preserve key order
    * outer: use union of keys from both frames, similar to a SQL full outer join; sort keys lexicographically
    * inner: use intersection of keys from both frames, similar to a SQL inner join; preserve the order of the left keys

In [ ]:
df6 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'],
                    'food': ['fish', 'beans', 'bread']},
                   columns=['name', 'food'])
df7 = pd.DataFrame({'name': ['Mary', 'Joseph'],
                    'drink': ['wine', 'beer']},
                   columns=['name', 'drink'])
print(df6); print(df7); pd.merge(df6, df7)

In [ ]:
pd.merge(df6, df7, how = 'inner')

In [ ]:
pd.merge(df6, df7, how = 'outer')

In [ ]:
pd.merge(df6, df7, how = 'left')

In [ ]:
pd.merge(df6, df7, how = 'right')

In [ ]:
pd.merge(df7, df6, how = 'left')

In [ ]:
pd.merge(df7, df6, how = 'right')

### Overlapping Column Names: The suffixes Keyword
* Two input dfs have conflicting column names.
* suffixes : 2-length sequence (tuple, list, ...)
    * Suffix to apply to overlapping column names in the left and right side, respectively

In [ ]:
df8 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [1, 2, 3, 4]})
df9 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [3, 1, 4, 2]})
print(df8); print(df9); pd.merge(df8, df9, on = 'name')
# Because the output would have two conflicting column names, 
# the merge function automatically appends a suffix_x or _y 
# to make output columns unique.

* Specify a customer suffic using the **suffixes** keyword.

In [ ]:
pd.merge(df8, df9, on = 'name', suffixes=['_df8', '_df9'])

### Example: US States Data

In [ ]:
# Following are shell commands to download the data
!curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-population.csv
!curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-areas.csv
!curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-abbrevs.csv

In [ ]:
pop = pd.read_csv('state-population.csv')
areas = pd.read_csv('state-areas.csv')
abbrevs = pd.read_csv('state-abbrevs.csv')

#### Rank US states and territories by their 2010 population density

In [ ]:
# use how='outer' to make sure no data is thrown away due to mismatched labels
merged = pd.merge(pop, abbrevs, how = 'outer',
                 left_on = 'state/region', right_on = 'abbreviation')
# drop duplicate ino
merged = merged.drop('abbreviation', 1) 
merged.head()

In [ ]:
merged.isnull().any()

In [ ]:
merged[ merged['population'].isnull() ]
# all the null population values are from Puerto Rico prior to the year
2000;

In [ ]:
# some of the new state entries are also null, 
# which means that there was no corresponding entry in the abbrevs key! 
# Let’s figure out which regions lack this match
merged.loc[merged['state'].isnull(), 'state/region'].unique()

In [ ]:
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'

In [ ]:
merged.isnull().any()

## 

In [ ]:
# all the null population values are from Puerto Rico prior to the year
2000;
merged.loc[merged['population'].isnull()]

In [ ]:
merged.loc[(merged['year'] < 2000) & (merged['state/region'] == 'PR')]

In [ ]:
# Join the areas df
final = pd.merge(merged, areas, on = 'state', how = 'left')
final.head(n = 20)

In [ ]:
# Check null
final.isnull().any()

state_pop_2010 = state_pop[state_pop['year']==2010 ]
state_pop_2010

In [ ]:
final.loc[final['area (sq. mi)'].isnull()]

In [ ]:
final.loc[:, 'area (sq. mi)'][final['area (sq. mi)'].isnull()]

In [ ]:
final['state'][final['area (sq. mi)'].isnull()].unique()

In [ ]:
usarea = final['area (sq. mi)'].dropna().unique().sum()

In [ ]:
final.loc[:, 'area (sq. mi)'][final['area (sq. mi)'].isnull()].fillna(usarea, inplace = True)

In [ ]:
final.isnull().any()

In [ ]:
final[final['state'] == 'United States']

In [ ]:
data2010 = final.query("year == 2010 & ages == 'total'")
data2010.head()

In [ ]:
data2010.set_index('state', inplace = True)
density = data2010['population'] / data2010['area (sq. mi)']

In [ ]:
density.sort_values(ascending = False, inplace = True)

In [ ]:
density.head(n = 15)